# M3.2 — Cloud Deployment (Railway/Render)

**Objective:** Deploy a FastAPI app to Railway and Render with honest reality checks.

**Platforms Covered:**
- Railway (fast, auto-detection)
- Render (more config, free tier)

**Prerequisites:**
- GitHub account
- Railway/Render account (free tier)
- Git repo with app code

## Section 1: Reality Check — Platform Capabilities & Limits

### What Railway & Render ACTUALLY Offer

**Railway:**
- ✅ Zero-config deployment (auto-detects Dockerfile/Procfile)
- ✅ Fast: GitHub push → live in <10 min
- ✅ Built-in PostgreSQL, Redis, MySQL
- ✅ Auto-generated public URLs
- ❌ Free tier: $5 monthly credit (~140 hours)
- ❌ Sleeps after 15 min inactivity

**Render:**
- ✅ 750 free hours/month (one always-on app)
- ✅ Auto-deploy from GitHub
- ✅ Free PostgreSQL (90-day limit)
- ❌ Manual env var config
- ❌ No free Redis (use Upstash)
- ❌ Cold start: 30-60s after sleep

### Critical Limitations

| Issue | Railway | Render |
|-------|---------|--------|
| Memory | 512MB | 512MB-1GB |
| Cold Start | 15 min idle | 15 min idle |
| VPC Access | Paid only | No |
| Persistent Storage | Limited | /opt/render/project |

**Deal Breakers:**
- Large ML models → OOM errors
- High traffic (>50K req/day) → upgrade required
- HIPAA/SOC2 compliance → use AWS/GCP
- Private networking → paid Railway or VPS

In [ ]:
# Sanity check: Verify local app works before deploying
print("SAVED_SECTION: 1")
print("✅ Section 1 Complete: Reality Check")

## Section 2: Prep Repo — .gitignore, env, secrets

### Step 2.1: Create .gitignore

In [ ]:
%%writefile .gitignore
# Python
__pycache__/
*.pyc
*.pyo
.env
.venv/
venv/

# IDE
.vscode/
.idea/

# OS
.DS_Store
Thumbs.db

# Secrets
*.key
*.pem
credentials.json

### Step 2.2: Validate Environment Variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Check critical env vars
required_vars = ["ADMIN_SECRET"]
missing = [v for v in required_vars if not os.getenv(v)]

if missing:
    print(f"⚠️  Missing: {missing}")
    print("Create .env from .env.example and set values")
else:
    print("✅ All required env vars present")

# Expected: ✅ All required env vars present

### Step 2.3: Test App Locally

In [ ]:
# Run in terminal (not this cell):
# python m3_2_deploy.py

# Then test endpoints:
# curl http://localhost:8000/health
# curl http://localhost:8000/ready

print("Expected:")
print('{"status": "healthy", "service": "m3.2-deploy"}')
print('{"status": "ready", "checks": {...}, "platform": "local"}')

In [ ]:
print("SAVED_SECTION: 2")
print("✅ Section 2 Complete: Repo Prep")

## Section 3: Render Deploy Steps

### Step 3.1: Push to GitHub

In [ ]:
# Run in terminal:
# git add .
# git commit -m "Add M3.2 deployment files"
# git push origin main

print("Expected:")
print("Enumerating objects: X, done.")
print("To github.com:user/repo.git")
print("   abc1234..def5678  main -> main")

### Step 3.2: Create Render Web Service

**Manual Steps (Render Dashboard):**

1. Go to https://dashboard.render.com/
2. Click **New +** → **Web Service**
3. Connect GitHub repo
4. Configure:
   - **Name:** m3-2-cloud-deploy
   - **Region:** Oregon (or closest)
   - **Branch:** main
   - **Build Command:** `pip install -r requirements.txt`
   - **Start Command:** `uvicorn m3_2_deploy:app --host=0.0.0.0 --port=$PORT`
5. **Environment:**
   - Add `PLATFORM=render`
   - Add `ADMIN_SECRET=<generate-random>`
6. Click **Create Web Service**

**Expected Output (Render Logs):**
```
==> Building...
==> Downloading Python 3.11
==> Installing dependencies
==> Starting service
INFO: Uvicorn running on http://0.0.0.0:10000
```

### Step 3.3: Verify Deployment

In [ ]:
# Test deployed service (replace URL)
# curl https://m3-2-cloud-deploy.onrender.com/health

print("Expected:")
print('{"status": "healthy", "service": "m3.2-deploy"}')
print("\nIf 502/503: Wait 30-60s for cold start")

In [ ]:
print("SAVED_SECTION: 3")
print("✅ Section 3 Complete: Render Deployment")

## Section 4: Railway Deploy Steps

### Step 4.1: Create Railway Project

**Manual Steps (Railway Dashboard):**

1. Go to https://railway.app/new
2. Click **Deploy from GitHub repo**
3. Select your repository
4. Railway auto-detects Python app
5. Click **Deploy Now**

**Expected Output (Railway Logs):**
```
#1 [builder] Nixpacks build
#2 Installing Python 3.11
#3 Running: pip install -r requirements.txt
#4 Starting: uvicorn m3_2_deploy:app
Deployment live at: https://m3-2-cloud-deploy.up.railway.app
```

### Step 4.2: Configure Environment Variables

**Railway Dashboard:**
1. Click your service → **Variables**
2. Add:
   - `PLATFORM=railway`
   - `ADMIN_SECRET=<generate-random>`
3. Click **Deploy** to restart

### Step 4.3: Generate Public Domain

In [ ]:
# Railway Dashboard: Settings → Networking → Generate Domain

# Test deployed service (replace URL)
# curl https://m3-2-cloud-deploy.up.railway.app/health

print("Expected:")
print('{"status": "healthy", "service": "m3.2-deploy"}')

In [ ]:
print("SAVED_SECTION: 4")
print("✅ Section 4 Complete: Railway Deployment")

## Section 5: Cold Starts, Always-On & Cost Math

### Cold Start Reality

**Free Tier Behavior:**
- Sleep after 15 minutes of inactivity
- First request: 30-60 second wake-up
- Subsequent requests: <100ms

**Solutions:**
1. **Upgrade to Paid** ($7-25/month) → always-on
2. **External Pinger** (cron-job.org, UptimeRobot) → keep warm
3. **Accept Trade-off** → free but slow first hit

In [ ]:
# Cost breakdown calculator
def calculate_monthly_cost(platform, tier, add_postgres=False, add_redis=False):
    costs = {
        "railway": {"free": 5, "hobby": 5, "pro": 20},
        "render": {"free": 0, "starter": 7, "pro": 25}
    }
    
    base = costs[platform][tier]
    if add_postgres:
        base += 7 if platform == "railway" else 7
    if add_redis:
        base += 5 if platform == "railway" else 0  # Render: use Upstash
    
    return base

# Example: Railway Hobby + PostgreSQL
cost = calculate_monthly_cost("railway", "hobby", add_postgres=True)
print(f"Railway Hobby + PostgreSQL: ${cost}/month")

# Example: Render Starter (no extras needed)
cost = calculate_monthly_cost("render", "starter")
print(f"Render Starter (always-on): ${cost}/month")

### When to Upgrade

| Traffic Level | Recommendation |
|---------------|----------------|
| <5K req/day | Free tier OK |
| 5-50K req/day | Starter ($7/mo) |
| 50-500K req/day | Pro ($25/mo) |
| >500K req/day | Migrate to AWS/GCP |

In [ ]:
print("SAVED_SECTION: 5")
print("✅ Section 5 Complete: Cost Analysis")

## Section 6: Alternatives — Fly.io, VPS, AWS/GCP Decision Flow

### Decision Tree

In [ ]:
def recommend_platform(budget, traffic_daily, compliance_needed, devops_skill):
    """
    Decision helper for platform selection
    
    Args:
        budget: Monthly budget in USD
        traffic_daily: Daily requests (approx)
        compliance_needed: bool (HIPAA, SOC2, etc.)
        devops_skill: 1-5 (1=beginner, 5=expert)
    """
    if compliance_needed:
        return "AWS/GCP (compliance certifications required)"
    
    if budget < 10:
        if traffic_daily < 5000:
            return "Railway/Render Free Tier (cold starts OK)"
        else:
            return "Fly.io ($1.94/mo, better free tier)"
    
    if budget < 50:
        return "Railway/Render Starter ($7-25/mo, always-on)"
    
    if budget < 300:
        if devops_skill >= 3:
            return "VPS (DigitalOcean/Linode, full control)"
        else:
            return "Railway/Render Pro"
    
    return "AWS/GCP (scale + advanced features)"

# Examples
print(recommend_platform(budget=0, traffic_daily=1000, compliance_needed=False, devops_skill=2))
print(recommend_platform(budget=100, traffic_daily=50000, compliance_needed=False, devops_skill=4))
print(recommend_platform(budget=500, traffic_daily=100000, compliance_needed=True, devops_skill=5))

### Platform Comparison

| Platform | Best For | Starting Cost | Complexity |
|----------|----------|---------------|------------|
| Railway | Quick prototypes | $5 credit | Low |
| Render | Always-on free tier | $0-7/mo | Low |
| Fly.io | Global edge apps | $1.94/mo | Medium |
| VPS | Full control | $5-20/mo | High |
| AWS/GCP | Enterprise scale | $50+/mo | Very High |

In [ ]:
print("SAVED_SECTION: 6")
print("✅ Section 6 Complete: Alternatives Analysis")

## Section 7: Troubleshooting

### Common Issues & Fixes

#### Issue 1: App Crashes on Startup

In [ ]:
# Check logs in platform dashboard
# Railway: Deployments → View Logs
# Render: Logs tab

# Common causes:
# 1. Missing dependencies in requirements.txt
# 2. Wrong start command
# 3. Port binding (must use $PORT)

print("Debug checklist:")
print("1. Verify requirements.txt includes all imports")
print("2. Check start command uses $PORT or ${PORT}")
print("3. Ensure app listens on 0.0.0.0, not localhost")

#### Issue 2: Health Check Fails

In [ ]:
# Test health endpoint locally first
# curl http://localhost:8000/health

# If local works but platform fails:
# 1. Check healthCheckPath in config
# 2. Increase healthcheckTimeout (default: 30s)
# 3. Verify endpoint returns 200 status

print("Fix: Update render.yaml or railway.json")
print('healthCheckPath: /health')
print('healthcheckTimeout: 60  # increase if slow')

#### Issue 3: Environment Variables Not Loading

In [ ]:
# Check /env-check endpoint
# curl https://your-app.onrender.com/env-check

print("Expected output:")
print('{"env_vars_present": {"ADMIN_SECRET": true, "PORT": true}}')
print("\nIf false: Add variables in platform dashboard")
print("Railway: Variables tab")
print("Render: Environment tab")

#### Issue 4: 502 Bad Gateway

In [ ]:
# Causes:
# 1. Cold start (wait 30-60s)
# 2. App crashed (check logs)
# 3. Wrong port binding

print("Fix port binding:")
print("# m3_2_deploy.py")
print('port = int(os.getenv("PORT", "8000"))')
print('uvicorn.run(app, host="0.0.0.0", port=port)')

#### Issue 5: Deployment Takes Too Long

In [ ]:
# Optimize build:
# 1. Pin dependency versions (avoid resolving)
# 2. Use build cache (automatic on Railway/Render)
# 3. Remove unused dependencies

print("Expected build time:")
print("First build: 3-5 minutes")
print("Cached builds: 1-2 minutes")
print("\nIf >10 min: Check for network issues or large dependencies")

### Getting Help

**Railway:**
- Docs: https://docs.railway.app
- Discord: https://discord.gg/railway

**Render:**
- Docs: https://render.com/docs
- Community: https://community.render.com

In [ ]:
print("SAVED_SECTION: 7")
print("✅ Section 7 Complete: Troubleshooting")
print("\n🎉 M3.2 Cloud Deployment Notebook Complete!")